# IBM Data Scienze Professional Certificate Capstone

## Segmenting and Clustering Neighborhoods in Toronto

In this project, we will collect Toronto neighborhoods data from the wiki page, convert it to a pandas dataframe, clean and pre-process it. We will then collect the coordinates of each neighborhood and add them to the dataframe, so that we can use Foursquare location data. 

Finally, we will explore and cluster Toronto neighborhoods.

### Install packages

In [1]:

import sys
!{sys.executable} -m pip install pandas
!{sys.executable} -m pip install beautifulsoup4
!{sys.executable} -m pip install requests
!{sys.executable} -m pip install geopy
!{sys.executable} -m pip install sklearn
!{sys.executable} -m pip install folium
!{sys.executable} -m pip install matplotlib


  Created wheel for sklearn: filename=sklearn-0.0-py2.py3-none-any.whl size=1321 sha256=081c9a44d89723a54198dfd23da2f0317baa96c0997d502a522b2567b16cb793
  Stored in directory: C:\Users\lauro\AppData\Local\pip\Cache\wheels\76\03\bb\589d421d27431bcd2c6da284d5f2286c8e3b2ea3cf1594c074
Successfully built sklearn


In [2]:
from bs4 import BeautifulSoup
from geopy.geocoders import Nominatim
from sklearn.cluster import KMeans
import requests
import pandas as pd
from pandas.io.json import json_normalize
import numpy as np
import folium
import matplotlib.cm as cm
import matplotlib.colors as colors

## 1. Collect Toronto neighborhood data and convert to DataFrame

The data is collected from https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M

In [3]:
# Get the HTML text from the wiki page
wiki_url = 'https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
html_text = requests.get(wiki_url).text

# Extract the table data using BeautifulSoup
soup = BeautifulSoup(html_text)
table = soup.find('table', attrs={'class':'wikitable sortable'})
trs = table.find_all('tr')

# Extract the text from all the table cells and add all rows
# to a list of rows.
rows = list()
for tr in trs:
    td = tr.find_all('td')
    row = [ele.text.strip() for ele in td]
    if row:
        # Ignore empty rows with no 'td',
        # applicable for the column headers row.
        rows.append(row)
        
# Convert the data to a pandas dataframe with 
# columns 'PostalCode', 'Borough', and 'Neighborhood'.
df = pd.DataFrame(rows,
                  columns=['PostalCode', 'Borough', 'Neighborhood'])

# Remove rows with column 'Not assigned' for column 'Borough'
df = df[df.Borough != 'Not assigned']
df.reset_index(inplace=True, drop=True)

# If a cell has a borough but a Not assigned neighborhood, 
# then the neighborhood will be the same as the borough.
# So, replace 'Neighborhood' columns with value as
# 'Not assigned' with the value of its 'Borough'
df['Neighborhood'] = df.apply(
    lambda row: 
    row['Borough'] if row['Neighborhood'] == 'Not assigned' 
    else row['Neighborhood'],
    axis=1)

# More than one neighborhood can exist in one postal code area.
# Combine rows with the same postal code into a single row.
# Separate neighborhoods with same postal code by ','
df = df.groupby(['PostalCode', 'Borough'])['Neighborhood'].\
    apply(', '.join).to_frame()
df.reset_index(inplace=True)

In [7]:
df.head()

,PostalCode,Borough,Neighborhood
0,M1B,Scarborough,"Malvern, Rouge"
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


In [8]:
df.shape

(103, 3)

## 2. Add coordinates data to df

We use eospatial_data to get latitude and longitude, and concatenate the data with our initial dataframe.

In [9]:
geospatial_data = pd.read_csv('https://cocl.us/Geospatial_data')

## Combine the two dataframes.

# Concatenate the two dataframes using column "PostalCode" from 'df' and
# "Postal Code" from 'geospatial_data'.
df = pd.concat(
    [df.set_index('PostalCode'), geospatial_data.set_index('Postal Code')],
    axis=1, join='inner')

# Postal code will now be the index, let's reset it
df.reset_index(inplace=True)

# Rename the 'index' column to 'PostalCode'
df.rename(columns={'index': 'PostalCode'}, inplace=True)

In [11]:
df.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
1,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476


In [14]:
df.shape

(103, 5)

## 3. Semgenting and Clustering Toronto neighborhoods

We begin by creating a map of Toronto.

In [15]:
# get Toronto coords
address = 'Toronto, Canada'
geolocator = Nominatim(user_agent="toronto_explorer")
location = geolocator.geocode(address)
print('The geograpical coordinate of Toronto are {}, {}.'.format(
    location.latitude, location.longitude))

The geograpical coordinate of Toronto are 43.6534817, -79.3839347.


In [16]:
# create a map using Folium
map_toronto = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=10)
map_toronto

We can now superimpose Toronto's neighborhoods on the map

In [17]:
#Markers are added to the map for each of the neighborhoods 
#and each marker is labeled in the format: "PostalCode (Neighborhood), Borough"
for _, row in df.iterrows():
    label = '{} ({}), {}'.format(
        row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

We should limit the dataset to boroughs containing 'Toronto' in their names

In [19]:
# Create dataframe with boroughs containing the term 'Toronto'
toronto_data = df[df.Borough.str.contains('Toronto')]
toronto_data.reset_index(inplace=True, drop=True)
toronto_data.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"India Bazaar, The Beaches West",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,"North Toronto West, Lawrence Park",43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Summerhill West, Rathnelly, South Hill, Forest...",43.686412,-79.400049


In [21]:
toronto_data.shape

(39, 5)

In [22]:
# let's visualise them on the map
# Create a map instance
map_toronto = folium.Map(
    location=[location.latitude, location.longitude],
    zoom_start=11)

# Add markers for all 38 neighborhoods
for _, row in toronto_data.iterrows():
    label = '{} ({}), {}'.format(
        row.PostalCode, row.Neighborhood, row.Borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [row.Latitude, row.Longitude],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto) 
    
map_toronto

We can now use Foursquare to explore the neighborhoods in our dataframe

In [24]:
CLIENT_ID = 'JQ5BH2SUTJZAOIB3VU2VXAJUZGUHWERBKQOMQ2DUP1KTVFGZ' # your Foursquare ID
CLIENT_SECRET = 'BIPNUM1E1FFIAOBQDLFL4VA3LNRXALDHMTVWY0V1WAGEZ2WX'
VERSION = '20190601'

neighborhood_name = toronto_data.loc[0, 'Neighborhood']
neighborhood_latitude = toronto_data.loc[0, 'Latitude']
neighborhood_longitude = toronto_data.loc[0, 'Longitude']

print("Latitude and longitude of neighborhood '{}' are [{}, {}]".format(
    neighborhood_name, neighborhood_latitude, neighborhood_longitude))

Latitude and longitude of neighborhood 'The Beaches' are [43.67635739999999, -79.2930312]


Let's get the top 100 venues in The Beaches within a radius of 500 metres

In [25]:
limit = 100
radius = 500
explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, 
    neighborhood_latitude, neighborhood_longitude, radius, limit)

In [27]:
# Get the venues.
results = requests.get(url).json()


In [28]:
# Explore the venues
venues = results['response']['groups'][0]['items']
venues

[{'reasons': {'count': 0,
   'items': [{'summary': 'This spot is popular',
     'type': 'general',
     'reasonName': 'globalInteractionReason'}]},
  'venue': {'id': '4bd461bc77b29c74a07d9282',
   'name': 'Glen Manor Ravine',
   'location': {'address': 'Glen Manor',
    'crossStreet': 'Queen St.',
    'lat': 43.67682094413784,
    'lng': -79.29394208780985,
    'labeledLatLngs': [{'label': 'display',
      'lat': 43.67682094413784,
      'lng': -79.29394208780985}],
    'distance': 89,
    'cc': 'CA',
    'city': 'Toronto',
    'state': 'ON',
    'country': 'Canada',
    'formattedAddress': ['Glen Manor (Queen St.)', 'Toronto ON', 'Canada']},
   'categories': [{'id': '4bf58dd8d48988d159941735',
     'name': 'Trail',
     'pluralName': 'Trails',
     'shortName': 'Trail',
     'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/parks_outdoors/hikingtrail_',
      'suffix': '.png'},
     'primary': True}],
   'photos': {'count': 0, 'groups': []}},
  'referralId': 'e-0-4bd461bc77b2

In [29]:
# Normalize the JSON response
neighborhood_venues = json_normalize(venues)
neighborhood_venues


,referralId,reasons.count,reasons.items,venue.id,venue.name,venue.location.address,venue.location.crossStreet,venue.location.lat,venue.location.lng,venue.location.labeledLatLngs,...,venue.location.cc,venue.location.city,venue.location.state,venue.location.country,venue.location.formattedAddress,venue.categories,venue.photos.count,venue.photos.groups,venue.location.postalCode,venue.venuePage.id
0,e-0-4bd461bc77b29c74a07d9282-0,0,"[{'summary': 'This spot is popular', 'type': '...",4bd461bc77b29c74a07d9282,Glen Manor Ravine,Glen Manor,Queen St.,43.676821,-79.293942,"[{'label': 'display', 'lat': 43.67682094413784...",...,CA,Toronto,ON,Canada,"[Glen Manor (Queen St.), Toronto ON, Canada]","[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",0,[],NaN,NaN
1,e-0-4ad4c062f964a52011f820e3-1,0,"[{'summary': 'This spot is popular', 'type': '...",4ad4c062f964a52011f820e3,The Big Carrot Natural Food Market,125 Southwood Dr,NaN,43.678879,-79.297734,"[{'label': 'display', 'lat': 43.678879, 'lng':...",...,CA,Toronto,ON,Canada,"[125 Southwood Dr, Toronto ON M4E 0B8, Canada]","[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",0,[],M4E 0B8,75150878
2,e-0-4b8daea1f964a520480833e3-2,0,"[{'summary': 'This spot is popular', 'type': '...",4b8daea1f964a520480833e3,Grover Pub and Grub,676 Kingston Rd.,at Main St.,43.679181,-79.297215,"[{'label': 'display', 'lat': 43.67918143494101...",...,CA,Toronto,ON,Canada,"[676 Kingston Rd. (at Main St.), Toronto ON M4...","[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",0,[],M4E 1R4,NaN
3,e-0-4df91c4bae60f95f82229ad5-3,0,"[{'summary': 'This spot is popular', 'type': '...",4df91c4bae60f95f82229ad5,Upper Beaches,NaN,NaN,43.680563,-79.292869,"[{'label': 'display', 'lat': 43.68056321147582...",...,CA,Toronto,ON,Canada,"[Toronto ON, Canada]","[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",0,[],NaN,NaN


There are 4 venues in The Beaches. Let's explore further.

In [31]:
# Filter out the venue name, category, latitude and logitude.
venue_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
neighborhood_venues = neighborhood_venues.loc[:, venue_columns]
neighborhood_venues

,venue.name,venue.categories,venue.location.lat,venue.location.lng
0,Glen Manor Ravine,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.676821,-79.293942
1,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
2,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
3,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869


In [33]:

# Change the column names to just the last part after the '.'.
neighborhood_venues.columns = [column.split(".")[-1] for column in neighborhood_venues.columns]
neighborhood_venues

,name,categories,lat,lng
0,Glen Manor Ravine,"[{'id': '4bf58dd8d48988d159941735', 'name': 'T...",43.676821,-79.293942
1,The Big Carrot Natural Food Market,"[{'id': '50aa9e744b90af0d42d5de0e', 'name': 'H...",43.678879,-79.297734
2,Grover Pub and Grub,"[{'id': '4bf58dd8d48988d11b941735', 'name': 'P...",43.679181,-79.297215
3,Upper Beaches,"[{'id': '4f2a25ac4b909258e854f55f', 'name': 'N...",43.680563,-79.292869


In [34]:
# Extract the category names from a row.
# We will get the first item in the categories list and then get its name.

def get_category(row):
    categories_list = row['categories']
    if categories_list:
        return categories_list[0]['name']
    return None

In [35]:
# Replace the values in categories column with the first catogory name.
neighborhood_venues['categories'] = neighborhood_venues.apply(get_category, axis=1)
neighborhood_venues

,name,categories,lat,lng
0,Glen Manor Ravine,Trail,43.676821,-79.293942
1,The Big Carrot Natural Food Market,Health Food Store,43.678879,-79.297734
2,Grover Pub and Grub,Pub,43.679181,-79.297215
3,Upper Beaches,Neighborhood,43.680563,-79.292869


We can now explore all the neighborhoods as we did with the Beaches

In [36]:
venues_list = list()

for name, lat, lng in zip(toronto_data['Neighborhood'], toronto_data['Latitude'], toronto_data['Longitude']):
    print("Collecting venues for neighborhood:", name)
    
    # Create API request URL
    limit = 100
    radius = 500
    explore_url_prefix = 'https://api.foursquare.com/v2/venues/explore'
    url = '{}?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
        explore_url_prefix, CLIENT_ID, CLIENT_SECRET, VERSION, 
        lat, lng, radius, limit)
    
    # Make the request
    neighborhood_venues = requests.get(url).json()["response"]['groups'][0]['items']
    
    # Add relevant info to venues_list
    venues_list.extend([(
        name, lat, lng,
        v['venue']['name'], 
        v['venue']['location']['lat'], 
        v['venue']['location']['lng'],  
        v['venue']['categories'][0]['name']) for v in neighborhood_venues])

print("Done")


Done


In [37]:
# Let us see how many venues we have got.
len(venues_list)


1634

In [38]:
# Convert the venues_list to a dataframe.
toronto_venues = pd.DataFrame(venues_list)
toronto_venues.columns = [
    'Neighborhood', 'Neighborhood Latitude', 'Neighborhood Longitude', 
    'Venue', 'Venue Latitude', 'Venue Longitude', 'Venue Category']

In [39]:
toronto_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood
4,"The Danforth West, Riverdale",43.679557,-79.352188,MenEssentials,43.677820,-79.351265,Cosmetics Shop


In [40]:
#check number of venues in each neighborhood
toronto_venues.groupby('Neighborhood').count()


,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"Brockton, Parkdale Village, Exhibition Place",25,25,25,25,25,25
"Business reply mail Processing Centre, South Central Letter Processing Plant Toronto",19,19,19,19,19,19
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",15,15,15,15,15,15
Central Bay Street,63,63,63,63,63,63
Christie,17,17,17,17,17,17
Church and Wellesley,75,75,75,75,75,75
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
Davisville,33,33,33,33,33,33


In [41]:
# check unique venues categories
print('There are {} uniques categories'.format(
    len(toronto_venues['Venue Category'].unique())))


There are 233 uniques categories
